In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from plot_utils import plot_surface
import QAOA_paper_proxy

%matplotlib widget

def test_plot_number_with_cost(num_constraints: int, num_qubits: int) -> None:
    costs = np.linspace(0, num_constraints, num_constraints+1)
    num_with_costs = np.zeros(len(costs))
    for i in range (len(costs)):
        num_with_costs[i] = QAOA_paper_proxy.number_with_cost_paper_proxy(costs[i], num_constraints, num_qubits)
    
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(costs, num_with_costs)
    
def test_plot_by_cost_2(cost: int, num_constraints: int, num_qubits: int) -> None:
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_paper_proxy.number_of_costs_at_distance_paper_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits)
    
    plot_surface(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from paper", "cost_2", "distance", "proxy")

test_plot_number_with_cost(20, 10)
test_plot_by_cost_2(4, 50, 12)
plt.show()

In [ ]:
from QAOA_paper_proxy import QAOA_paper_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations


gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(10, 6, gammas, betas)
plot_heat_map(gammas, betas, overlaps, "Expectation Proxy from paper", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_proxy import QAOA_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray):
    expectations = np.zeros([len(gammas), len(betas)])
    amplitudes = np.zeros([num_constraints + 1, len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            amps, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits)
            for c in range(num_constraints+1):
                amplitudes[c][i][j] = abs(amps[1][c]) ** 2

    return amplitudes, expectations


gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
amplitudes, overlaps = collect_parameter_data(21, 10, gammas, betas)

for c in range(0, 21):
    plot_heat_map(gammas, betas, amplitudes[c], f"Amplitude for c={c}", "Gamma", "Beta")

plot_heat_map(gammas, betas, overlaps, f"Expectation Proxy", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_paper_proxy import QAOA_paper_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np


def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray):
    expectations = np.zeros([len(gammas), len(betas)])
    amplitudes = np.zeros([num_constraints + 1, len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            amps, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)
            for c in range(num_constraints+1):
                amplitudes[c][i][j] = abs(amps[1][c]) ** 2

    return amplitudes, expectations


gammas = np.linspace(0, np.pi/2, 25)
betas = np.linspace(0, np.pi, 25)
amplitudes, overlaps = collect_parameter_data(21, 10, gammas, betas)

for c in range(0, 21):
    plot_heat_map(gammas, betas, amplitudes[c], f"Amplitude for c={c}", "Gamma", "Beta")

plot_heat_map(gammas, betas, overlaps, f"Expectation Proxy", "Gamma", "Beta")

plt.show()

In [ ]:
from QAOA_simulator import get_simulator, get_overlap, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)

    return overlaps

N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, overlaps, "True QAOA Overlap", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_expectation, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)

    return expectations

N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, overlaps, "True Expectation", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_expectation, get_overlap, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np

def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)


    return expectations, overlaps

N = 15
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 21 edges
gammas = np.linspace(0, np.pi/2, 50)
betas = np.linspace(0, np.pi, 50)
expectations, overlaps = collect_parameter_data(G, gammas, betas)
shifted_expectations = expectations - np.amin(expectations)
_ = plot_heat_map(gammas, betas, expectations, "True Expectation", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, overlaps, "True Overlap", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, (shifted_expectations/np.amax(shifted_expectations)) - (overlaps/np.amax(overlaps)), "True Expectation Minus Overlap", "Gamma", "Beta")

In [ ]:
from QAOA_simulator import get_simulator, get_overlap, get_expectation, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np
from QAOA_proxy import QAOA_proxy
from QAOA_paper_proxy import QAOA_paper_proxy


def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    overlaps = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            overlaps[i][j] = get_overlap(N, ising_model, gamma, beta, sim, result)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)

    return expectations, overlaps


def collect_parameter_data_proxy_less_dumb(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray, h: float, hc: float, l: float, r: float) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    altered_proxy = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits, h, hc, l, r)
            _, altered_proxy[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits, h, hc, l, r, 2*(num_constraints+1)//3)

    return expectations, altered_proxy


def collect_parameter_data_proxy(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits, 2*(num_constraints+1)//3)

    return expectations


N = 10
G = nx.erdos_renyi_graph(N, 0.5, seed=42)  # has 10 edges
gammas = np.linspace(0, np.pi, 50)
betas = np.linspace(0, 2*np.pi, 50)
expectations, overlaps = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, expectations, "True QAOA Expectations", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, overlaps, "True QAOA Overlap", "Gamma", "Beta")
#expectation_proxies = collect_parameter_data_proxy(G.number_of_edges(), N, gammas, betas)
#_ = plot_heat_map(gammas, betas, expectation_proxies, "Expectation Proxies from Paper", "Gamma", "Beta")
expectation_proxies_less_dumb, altered_less_dumb = collect_parameter_data_proxy_less_dumb(G.number_of_edges(), N, gammas, betas, 0, 0, 1, 1)
_ = plot_heat_map(gammas, betas, expectation_proxies_less_dumb, "Expectation Proxies from us", "Gamma", "Beta")
_ = plot_heat_map(gammas, betas, altered_less_dumb, "Altered Proxies from us", "Gamma", "Beta")

In [ ]:
import QAOA_proxy as qpld
import numpy as np


def test_plot(num_constraints: int, num_qubits: int) -> None:
    costs = np.linspace(0, num_constraints, num_constraints + 1)
    num_with_costs = np.zeros(len(costs))
    for i in range(len(costs)):
        num_with_costs[i] = qpld.number_with_cost_proxy(costs[i], num_constraints, num_qubits)

    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(costs, num_with_costs)


test_plot(20, 10)

In [ ]:
import QAOA_proxy
import numpy as np
from plot_utils import plot_surface, animate_3d_rotation
import QAOA_paper_proxy
import matplotlib.animation as animation

%matplotlib widget

def test_plot_by_cost_paper(cost: int, num_constraints: int, num_qubits: int):
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_paper_proxy.number_of_costs_at_distance_paper_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits)
    
    return plot_surface(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from paper", "cost_2", "distance", "proxy")

def test_plot_by_cost(cost: int, num_constraints: int, num_qubits: int) -> None:
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_proxy.number_of_costs_at_distance_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits, 0, 0, 1, 1)
    
    return plot_surface(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) from us", "cost_2", "distance", "proxy")

fig, ax = test_plot_by_cost_paper(8, 20, 10)
fig2, ax2 = test_plot_by_cost(8, 20, 10)

animate_3d_rotation(fig, ax, 'rotation1.gif')
animate_3d_rotation(fig2, ax2, 'rotation2.gif')

In [ ]:
from QAOA_paper_proxy import number_of_costs_at_distance_paper_proxy
import numpy as np
from plot_utils import plot_3d_scatter

%matplotlib widget

def test_plot(num_constraints_max: int, num_qubits_max: int) -> None:
    num_constraints = np.linspace(0, num_constraints_max, num_constraints_max + 1, dtype=int)
    num_qubits = np.linspace(0, num_qubits_max, num_qubits_max + 1, dtype=int)
    peak_hs = np.zeros([len(num_constraints), len(num_qubits)])
    for c in range(len(num_constraints)):
        for q in range(len(num_qubits)):
            try:
                peak_hs[c][q] = number_of_costs_at_distance_paper_proxy(c//2, c//2, q//2, c, q)
            except:
                ()
    
    plot_3d_scatter(num_constraints[7:], num_qubits, peak_hs[7:], f"Peak values from Paper", "num_constraints", "num_qubits", "peak value")

test_plot(15, 12)

In [ ]:
import numpy as np
from plot_utils import plot_3d_scatter

%matplotlib widget

def test_plot(num_constraints_max: int, num_qubits_max: int) -> None:
    num_constraints = np.linspace(0, num_constraints_max, num_constraints_max + 1, dtype=int)
    num_qubits = np.linspace(0, num_qubits_max, num_qubits_max + 1, dtype=int)
    peak_hs = np.zeros([len(num_constraints), len(num_qubits)])
    for c in range(len(num_constraints)):
        for q in range(len(num_qubits)):
            try:
                peak_hs[c][q] = 2**(q-4)
            except:
                ()
    plot_3d_scatter(num_constraints, num_qubits, peak_hs, f"Peak values from us", "num_constraints", "num_qubits", "peak value_proxy")

test_plot(15, 12)

In [ ]:
import networkx as nx

N = 10
nx.erdos_renyi_graph(N, 0.5, seed=42).number_of_edges()

In [1]:
from QAOA_simulator import get_expectation, get_simulator, get_overlap, QAOA_run
import networkx as nx
import qokit.maxcut as mc
import numpy as np
import QAOA_proxy as qp
import QAOA_proxy_optimizer as qpo
#import QAOA_paper_proxy as qpp
from itertools import product


N = 10
p = 2
G = nx.erdos_renyi_graph(N, 0.5, seed = 42)
num_edges = G.number_of_edges()
ising_model = mc.get_maxcut_terms(G)
sim = get_simulator(N, ising_model)
print(f"Max Cut: {max(sim.get_cost_diagonal())}")
init_gamma, init_beta = np.full((2, p), 0.1)

real_result = QAOA_run(ising_model, N, p, init_gamma, init_beta)
real_gamma = real_result["gamma"]
real_beta = real_result["beta"]
real_expectation = real_result["expectation"]
real_overlap = real_result["overlap"]
print(f"g: {real_gamma}, b: {real_beta}, e: {real_expectation}, o: {real_overlap}")

pre_result = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta)
pre_gamma = pre_result["gamma"]
pre_beta = pre_result["beta"]
pre_expectation = get_expectation(N, ising_model, pre_gamma, pre_beta)
pre_overlap = get_overlap(N, ising_model, pre_gamma, pre_beta)
print(f"g: {pre_gamma}, b: {pre_beta}, e: {pre_expectation}, o: {pre_overlap}")

init_h_tweaks = [1]
init_hc_tweaks = [0, 2]
init_l_tweaks = [0.1, 1, 2]
init_r_tweaks = [0.1, 1, 2]
init_tweaks = product(init_h_tweaks, init_hc_tweaks, init_l_tweaks, init_r_tweaks)

h, hc, l, r = qpo.QAOA_proxy_optimize(num_edges, N, p, init_gamma, init_beta, init_tweaks=init_tweaks)
print(f"h: {h}\nhc: {hc}\nl: {l}\nr: {r}")
result = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h, hc_tweak_add=hc, l_tweak_mul=l, r_tweak_mul=r)
gamma = result["gamma"]
beta = result["beta"]
expectation = get_expectation(N, ising_model, gamma, beta)
approx_ratio = expectation/max(sim.get_cost_diagonal())
overlap = get_overlap(N, ising_model, gamma, beta)
print(f"g: {gamma}, b: {beta}, e: {expectation}, o: {overlap}")

print("Start")
for _ in range(10):
    G = nx.erdos_renyi_graph(N, 0.5)
    while (G.number_of_edges() != num_edges):
        G = nx.erdos_renyi_graph(N, 0.5)
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)
    print(f"MaxCut: {max(sim.get_cost_diagonal())}")
    print(f"e: {get_expectation(N, ising_model, gamma, beta, sim)}, o: {get_overlap(N, ising_model, gamma, beta, sim)}")


#result2 = qpp.QAOA_paper_proxy_run(G.number_of_edges(), N, p, init_gamma, init_beta)
#gamma2 = result2["gamma"]
#beta2 = result2["beta"]
#expectation2 = get_expectation(N, ising_model, gamma2, beta2)
#print(f"g: {gamma2}, b: {beta2}, e: {expectation2}")

/home/nkohen/dev/QOKit/grips/QAOA_proxy.py:26: JuliaWarning: USE_JULIA=False, so calling python version of QAOA_proxy. Did you mean to call QAOA_proxy_python?
  warnings.warn("USE_JULIA=False, so calling python version of QAOA_proxy. Did you mean to call QAOA_proxy_python?", JuliaWarning)


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Max Cut: 15.0
g: [0.60729518 1.23338023 1.51375319], b: [0.4580484  0.3055645  0.16623899], e: 14.230756860040874, o: 0.5042338171708334
g: [ 0.73097805 -1.23440413 -0.16696553], b: [1.94927137 1.19545912 1.1864342 ], e: 8.160270598645209, o: 0.009908793618320278
h: 1.9682688294810495
hc: 2.749016430460764
l: 1.829672431219189
r: 1.2590326485080765
g: [-0.53786195 -0.07879607  0.87280154], b: [1.2226664  1.22464704 1.92107174], e: 12.764466884853158, o: 0.23203200693946163
Start
MaxCut: 15.0
e: 12.653307664275037, o: 0.1531939914474503
MaxCut: 16.0
e: 13.096239534551717, o: 0.15835161216874102
MaxCut: 16.0
e: 12.640448563126089, o: 0.08046690432958996
MaxCut: 17.0
e: 12.908129585173207, o: 0.07149563920101255
MaxCut: 17.0
e: 12.963392677437799, o: 0.07876652679623507
MaxCut: 16.0
e: 12.86229569696382, o: 0.05167833392757955
MaxCut: 16.0
e: 12.356335673639405, o: 0.049483823

In [52]:
import QAOA_proxy_optimizer as qpo

p = 1
min_N = 7
max_N = 11

results = qpo.find_optimal_parameters(min_N, max_N, p)
print(results)

Starting work on N=7
Starting work on N=8
Starting work on N=9
Starting work on N=10
Starting work on N=11
{(7, 7): (0.9549164078817375, -0.3325977338650318, 0.36700660047014433, 0.13258108990636472), (7, 10): (1.9748841630806477, -0.08214825513554976, 0.5323824396365897, 0.5285241871845429), (7, 13): (3.138113957067362, -0.15437097378738132, 0.5854351318256218, 0.7759158603828146), (7, 16): (4.414902898902718, -0.2310997171851854, 0.71784933601411, 0.757088216860688), (8, 9): (2.650000220266842, 0.7255132680996125, 0.35075473386298206, 0.4406479530011581), (8, 12): (5.021534876051178, 0.8479393182103462, 0.7355018865021918, 0.857742274028321), (8, 15): (6.896667950146675, 0.8057615702568832, 0.6945760714226782, 1.074089432810694), (8, 18): (8.901369279127506, 0.9046661078783882, 0.6040782090404261, 1.3562013631194114), (8, 21): (11.204876132185376, 1.3159487924267992, 1.1089374463109019, 1.2438797362934753), (8, 24): (12.782783380222336, 1.0367384820100323, 1.4760474295959802, 1.11372

In [53]:
regr, score = qpo.linear_regression_for_parameters(results)
print(regr.coef_)
print(score)
regr.predict([[8, 21]])

[[ 2.29643743  0.69133414]
 [ 1.82774551  0.07680541]
 [ 0.2899717   0.08536894]
 [-0.44538214  0.11165447]]
0.8937303930158518


array([[11.07951147,  1.84440993,  1.32613731,  1.45585484]])

In [55]:
from QAOA_simulator import get_expectation, get_simulator, get_overlap
import QAOA_proxy as qp
import QAOA_paper_proxy as qpp
import qokit.maxcut as mc
import Graph_util
import numpy as np

p = 1
init_gamma, init_beta = np.full((2, p), 0.1)

min_N = 7
max_N = 11

orig_sum = 0
optimal_sum = 0
guess_sum = 0
rand_sum = 0
paper_sum = 0
rand_str_sum = 0
rand_str_smart_sum = 0
count = 0

for N in range(min_N, max_N + 1):
    max_edges = N*(N-1)//2
    for num_edges in range(max_edges // 3, max_edges - 3, 3):
        h, hc, l, r = results[N, num_edges]
        res = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h, hc_tweak_add=hc, l_tweak_mul=l, r_tweak_mul=r)
        gamma = res["gamma"]
        beta = res["beta"]
        guess = regr.predict(np.array([[N, num_edges]]))
        h_guess, hc_guess, l_guess, r_guess = guess[0][0], guess[0][1], guess[0][2], guess[0][3]
        res_guess = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h_guess, hc_tweak_add=hc_guess, l_tweak_mul=l_guess, r_tweak_mul=r_guess)
        gamma_guess = res_guess["gamma"]
        beta_guess = res_guess["beta"]
        gamma_rand, beta_rand = np.random.rand(2, p)*np.pi
        paper_res = qpp.QAOA_paper_proxy_run(num_edges, N, p, init_gamma, init_beta)
        gamma_paper = paper_res["gamma"]
        beta_paper = paper_res["beta"]
        res_orig = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta)
        gamma_orig = res_orig["gamma"]
        beta_orig = res_orig["beta"]
        for _ in range(40):
            G = Graph_util.erdos_renyi(N, num_edges)
            ising_model = mc.get_maxcut_terms(G)
            sim = get_simulator(N, ising_model)
            max_cut = max(sim.get_cost_diagonal())

            binary_array = np.hstack((np.ones(N//2), np.zeros(N - (N//2))))
            np.random.shuffle(binary_array)
            # Convert the numpy array to a string of binary digits
            binary_string = ''.join(binary_array.astype(int).astype(str))
            # Convert the binary string to an integer
            index = int(binary_string, 2)
            rand_str_smart = sim.get_cost_diagonal()[index]
            rand_str_smart_sum += rand_str_smart / max_cut

            rand_str_sum += np.random.choice(sim.get_cost_diagonal()) / max_cut
            optimal_e = get_expectation(N, ising_model, gamma, beta, sim)
            optimal_approx_ratio = optimal_e / max_cut
            guess_e = get_expectation(N, ising_model, gamma_guess, beta_guess, sim)
            guess_approx_ratio = guess_e / max_cut
            rand_e = get_expectation(N, ising_model, gamma_rand, beta_rand, sim)
            rand_approx_ratio = rand_e / max_cut
            paper_e = get_expectation(N, ising_model, gamma_paper, beta_paper, sim)
            paper_approx_ratio = paper_e / max_cut
            orig_e = get_expectation(N, ising_model, gamma_orig, beta_orig, sim)
            orig_approx_ratio = orig_e / max_cut
            orig_sum += orig_approx_ratio
            optimal_sum += optimal_approx_ratio
            guess_sum += guess_approx_ratio
            rand_sum += rand_approx_ratio
            paper_sum += paper_approx_ratio
            count += 1
            print(f"MaxCut ({N}, {num_edges}): {max_cut}")
            print(f"Orig:    e: {orig_e}, o: {get_overlap(N, ising_model, gamma_orig, beta_orig, sim)}, ar: {orig_approx_ratio}")
            print(f"Optimal: e: {optimal_e}, o: {get_overlap(N, ising_model, gamma, beta, sim)}, ar: {optimal_approx_ratio}")
            print(f"Guess:   e: {guess_e}, o: {get_overlap(N, ising_model, gamma_guess, beta_guess, sim)}, ar: {guess_approx_ratio}")
            print(f"Random:  e: {rand_e}, o: {get_overlap(N, ising_model, gamma_rand, beta_rand, sim)}, ar: {rand_approx_ratio}")
            print(f"Paper:   e: {paper_e}, o: {get_overlap(N, ising_model, gamma_paper, beta_paper, sim)}, ar: {paper_approx_ratio}")

print()
print(f"Count:        {count}")
print(f"Paper:        {paper_sum/count}")
print(f"Original:     {orig_sum/count}")
print(f"Optimal:      {optimal_sum/count}")
print(f"Guess:        {guess_sum/count}")
print(f"Random:       {rand_sum/count}")
print(f"RandomStr:    {rand_str_sum/count}")
print(f"RandStrSmart: {rand_str_smart_sum/count}")

MaxCut (7, 7): 6.0
Orig:    e: 4.695449107240281, o: 0.18755913147875242, ar: 0.7825748512067134
Optimal: e: 4.83821020463494, o: 0.23279963459917774, ar: 0.8063683674391567
Guess:   e: 4.803127879712239, o: 0.24329047145909094, ar: 0.8005213132853731
Random:  e: 2.1785132484028633, o: 0.008821029683474788, ar: 0.3630855414004772
Paper:   e: 4.325807133281005, o: 0.12629835905740183, ar: 0.7209678555468342
MaxCut (7, 7): 7.0
Orig:    e: 4.809483326047026, o: 0.10225632086213995, ar: 0.6870690465781466
Optimal: e: 5.031515915935014, o: 0.14188357507616986, ar: 0.7187879879907163
Guess:   e: 5.040379161610479, o: 0.15892012587370963, ar: 0.7200541659443542
Random:  e: 2.357397456622051, o: 0.005228257199333394, ar: 0.33677106523172157
Paper:   e: 4.370245251140199, o: 0.06001017343734326, ar: 0.6243207501628856
MaxCut (7, 7): 7.0
Orig:    e: 4.809483326047025, o: 0.10040976096618011, ar: 0.6870690465781465
Optimal: e: 5.031515915935014, o: 0.13198058685645664, ar: 0.7187879879907163
Gues

In [16]:
import QAOA_paper_proxy as qpp
from QAOA_simulator import get_expectation, get_simulator, get_overlap
import qokit.maxcut as mc
import Graph_util

N = 10
G = Graph_util.erdos_renyi(N, N*(N-1)//4)
p = 2
init_gamma, init_beta = np.full((2, p), 0.1)

result = qpp.QAOA_paper_proxy_run(G.number_of_edges(), N, p, init_gamma, init_beta)
print(result)

gamma = result["gamma"]
beta = result["beta"]
sim = get_simulator(N, mc.get_maxcut_terms(G))
print(max(sim.get_cost_diagonal()))
print(get_expectation(N, mc.get_maxcut_terms(G), gamma, beta, sim))
print(get_overlap(N, mc.get_maxcut_terms(G), gamma, beta, sim))

{'gamma': array([-0.24278705, -0.28953577]), 'beta': array([-0.33889322, -0.1402922 ]), 'expectation': 13.781234466694547, 'runtime': 1.4044764041900635, 'num_QAOA_calls': 164, 'classical_opt_success': True, 'scipy_opt_message': 'Optimization terminated successfully.'}
16.0
12.676717441494127
0.04162939174146642


In [4]:
N = 16
num_edges = 100
guess = regr.predict(np.array([[N, num_edges]]))
h_guess, hc_guess, l_guess, r_guess = guess[0][0], guess[0][1], guess[0][2], guess[0][3]
res_guess = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h_guess, hc_tweak_add=hc_guess, l_tweak_mul=l_guess, r_tweak_mul=r_guess)
gamma_guess = res_guess["gamma"]
beta_guess = res_guess["beta"]
for _ in range(5):
    G = Graph_util.erdos_renyi(N, num_edges)
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)
    max_cut = max(sim.get_cost_diagonal())
    guess_e = get_expectation(N, ising_model, gamma_guess, beta_guess, sim)
    guess_approx_ratio = guess_e / max_cut
    print(f"MaxCut ({N}, {num_edges}): {max_cut}")
    print(f"Guess:   e: {guess_e}, o: {get_overlap(N, ising_model, gamma_guess, beta_guess, sim)}, ar: {guess_approx_ratio}")

MaxCut (16, 100): 60.0
Guess:   e: 32.33248949974946, o: 4.498077563508541e-05, ar: 0.5388748249958243
MaxCut (16, 100): 60.0
Guess:   e: 31.911869197112978, o: 7.629152156282051e-05, ar: 0.5318644866185497
MaxCut (16, 100): 60.0
Guess:   e: 31.941794840869893, o: 8.96073442006026e-05, ar: 0.5323632473478316
MaxCut (16, 100): 60.0
Guess:   e: 32.22721682536932, o: 1.4385991554383033e-05, ar: 0.537120280422822
MaxCut (16, 100): 61.0
Guess:   e: 32.11080268077971, o: 1.828955263735511e-05, ar: 0.5264066013242575


In [16]:
from QAOA_simulator import QAOA_run

N = 16
num_edges = 100
guess = regr.predict(np.array([[N, num_edges]]))
h_guess, hc_guess, l_guess, r_guess = guess[0][0], guess[0][1], guess[0][2], guess[0][3]
res_guess = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta, h_tweak_sub=h_guess, hc_tweak_add=hc_guess, l_tweak_mul=l_guess, r_tweak_mul=r_guess)
gamma_guess = res_guess["gamma"]
beta_guess = res_guess["beta"]
for _ in range(2):
    G = Graph_util.erdos_renyi(N, num_edges)
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)
    max_cut = max(sim.get_cost_diagonal())
    print(f"MaxCut: {max_cut}")
    result_warm = QAOA_run(ising_model, N, p, gamma_guess, beta_guess, "Nelder-Mead")
    print(result_warm)
    print()
    init_gamma, init_beta = np.random.rand(2, p)
    result_cold = QAOA_run(ising_model, N, p, init_gamma, init_beta, "Nelder-Mead")
    print(result_cold)
    print()

MaxCut: 59.0
{'gamma': array([ 0.50725279, -0.07030606]), 'beta': array([-2.32236035,  2.58848055]), 'state': array([-0.00115064-0.00307481j, -0.00093144+0.00166761j,
        0.00011638+0.00071606j, ...,  0.00011638+0.00071606j,
       -0.00093144+0.00166761j, -0.00115064-0.00307481j]), 'expectation': 54.46964162861684, 'overlap': 0.011245020611886042, 'runtime': 177.243812084198, 'num_QAOA_calls': 357, 'classical_opt_success': True, 'scipy_opt_message': 'Optimization terminated successfully.'}

{'gamma': array([ 0.3484321 , -1.42977983]), 'beta': array([0.16365139, 1.44104156]), 'state': array([-0.00138694+2.14008765e-05j,  0.00032111+1.77718655e-03j,
       -0.00139223+5.05010768e-04j, ..., -0.00139223+5.05010768e-04j,
        0.00032111+1.77718655e-03j, -0.00138694+2.14008765e-05j]), 'expectation': 54.497738676845046, 'overlap': 0.011980109491886657, 'runtime': 114.30378150939941, 'num_QAOA_calls': 227, 'classical_opt_success': True, 'scipy_opt_message': 'Optimization terminated suc

In [51]:
from QAOA_simulator import get_expectation, get_simulator, get_overlap
import QAOA_proxy as qp
import QAOA_paper_proxy as qpp
import qokit.maxcut as mc
import Graph_util
import numpy as np

p = 1
init_gamma, init_beta = np.full((2, p), 0.1)

min_N = 15
max_N = 15

orig_sum = 0
rand_sum = 0
paper_sum = 0
rand_str_sum = 0
rand_str_smart_sum = 0
count = 0

for N in range(min_N, max_N + 1):
    max_edges = N*(N-1)//2
    for num_edges in [70]:
        gamma_rand, beta_rand = np.random.rand(2, p)*np.pi
        paper_res = qpp.QAOA_paper_proxy_run(num_edges, N, p, init_gamma, init_beta)
        gamma_paper = paper_res["gamma"]
        beta_paper = paper_res["beta"]
        res_orig = qp.QAOA_proxy_run(num_edges, N, p, init_gamma, init_beta)
        gamma_orig = res_orig["gamma"]
        beta_orig = res_orig["beta"]
        for _ in range(20):
            G = Graph_util.erdos_renyi(N, num_edges)
            ising_model = mc.get_maxcut_terms(G)
            sim = get_simulator(N, ising_model)
            max_cut = max(sim.get_cost_diagonal())

            binary_array = np.hstack((np.ones(N//2), np.zeros(N - (N//2))))
            np.random.shuffle(binary_array)
            # Convert the numpy array to a string of binary digits
            binary_string = ''.join(binary_array.astype(int).astype(str))
            # Convert the binary string to an integer
            index = int(binary_string, 2)
            rand_str_smart = sim.get_cost_diagonal()[index]
            rand_str_smart_sum += rand_str_smart / max_cut

            rand_str_sum += np.random.choice(sim.get_cost_diagonal()) / max_cut
            rand_e = get_expectation(N, ising_model, gamma_rand, beta_rand, sim)
            rand_approx_ratio = rand_e / max_cut
            paper_e = get_expectation(N, ising_model, gamma_paper, beta_paper, sim)
            paper_approx_ratio = paper_e / max_cut
            orig_e = get_expectation(N, ising_model, gamma_orig, beta_orig, sim)
            orig_approx_ratio = orig_e / max_cut
            orig_sum += orig_approx_ratio
            rand_sum += rand_approx_ratio
            paper_sum += paper_approx_ratio
            count += 1
            print(f"MaxCut ({N}, {num_edges}): {max_cut}")
            print(f"Orig:    e: {orig_e}, o: {get_overlap(N, ising_model, gamma_orig, beta_orig, sim)}, ar: {orig_approx_ratio}")
            print(f"Random:  e: {rand_e}, o: {get_overlap(N, ising_model, gamma_rand, beta_rand, sim)}, ar: {rand_approx_ratio}")
            print(f"Paper:   e: {paper_e}, o: {get_overlap(N, ising_model, gamma_paper, beta_paper, sim)}, ar: {paper_approx_ratio}")

print()
print(f"Count:        {count}")
print(f"Paper:        {paper_sum/count}")
print(f"Original:     {orig_sum/count}")
print(f"Random:       {rand_sum/count}")
print(f"RandomStr:    {rand_str_sum/count}")
print(f"RandStrSmart: {rand_str_smart_sum/count}")

MaxCut (15, 70): 46.0
Orig:    e: 38.17613526406956, o: 0.002488015593718405, ar: 0.8299159840015121
Random:  e: 34.99705039177359, o: 0.00033389537987546633, ar: 0.7608054432994259
Paper:   e: 37.209233655142896, o: 0.0005331311675879868, ar: 0.8088963838074542
MaxCut (15, 70): 45.0
Orig:    e: 38.12423729778921, o: 0.0028848377734844994, ar: 0.8472052732842046
Random:  e: 34.960416121443345, o: 0.0009290176944147865, ar: 0.7768981360320744
Paper:   e: 37.202237398030846, o: 0.000707553592754126, ar: 0.8267163866229077
MaxCut (15, 70): 47.0
Orig:    e: 38.02711345843015, o: 0.0015598707413181192, ar: 0.8090875203921308
Random:  e: 34.98257263217467, o: 9.718499861653417e-05, ar: 0.7443100560037165
Paper:   e: 37.18753440019863, o: 0.0003011750200118498, ar: 0.7912241361744389
MaxCut (15, 70): 47.0
Orig:    e: 37.82863041109087, o: 0.0031380449798405494, ar: 0.8048644768317207
Random:  e: 34.97735172912151, o: 0.00044445205303187514, ar: 0.7441989729600321
Paper:   e: 37.15516909849223

In [ ]:
import QAOA_proxy
import numpy as np
from plot_utils import plot_surface, animate_3d_rotation, plot_3d_scatter
import matplotlib.animation as animation

%matplotlib widget

def test_plot_by_cost(cost: int, num_constraints: int, num_qubits: int, h: float, hc: float, l: float, r: float) -> None:
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_proxy.number_of_costs_at_distance_proxy(cost, cost_2s[i], distances[j], num_constraints, num_qubits, h, hc, l, r)
    
    return plot_surface(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) for {(round(h, 2), round(hc, 2), round(l, 2), round(r, 2))}", "cost_2", "distance", "proxy")

i=0
def make_plot_and_save(h, hc, l, r):
    global i
    fig, ax = test_plot_by_cost(15, 30, 10, h, hc, l, r)
    animate_3d_rotation(fig, ax, f"rot{i}.gif")
    i+=1

make_plot_and_save(0, 0, 3, 3)
make_plot_and_save(0, 3, 1, 1)
make_plot_and_save(58, 0, 1, 1)
make_plot_and_save(0, 0, 1, 1)

In [ ]:
import QAOA_norm_proxy
import numpy as np
from plot_utils import plot_surface, animate_3d_rotation, plot_3d_scatter
import matplotlib.animation as animation

%matplotlib widget

def test_plot_by_cost(cost: int, num_constraints: int, num_qubits: int, mean: float, cov1: float, cov2: float):
    cost_2s = np.linspace(0, num_constraints, num_constraints+1, dtype=int)
    distances = np.linspace(0, num_qubits, num_qubits+1, dtype=int)
    num_of_costs_at_distances = np.zeros([len(cost_2s), len(distances)])
    for i in range(len(cost_2s)):
        for j in range(len(distances)):
            num_of_costs_at_distances[i][j] = QAOA_norm_proxy.number_of_costs_at_distance_norm_proxy(cost, cost_2s[i], distances[j], num_qubits, mean, cov1, cov2)
    
    return plot_surface(cost_2s, distances, num_of_costs_at_distances, f"\"Approximate\" N({cost};d,c) for {(round(mean, 2), round(cov1, 2), round(cov2, 2))}", "cost_2", "distance", "proxy")

fig, ax = test_plot_by_cost(29, 30, 10, 10, 20, 1)
animate_3d_rotation(fig, ax, "temp.gif")